In [1]:
if __name__=='__main__':
    import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [3]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='australia18.db'
    conn = sqlite3.connect(dbname)
    rally='Australia'

In [4]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [5]:
if __name__=='__main__':
    q='SELECT * FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'
    dd=pd.read_sql(q,conn)
    dd.head()

In [6]:
if __name__=='__main__':
    dd[dd['name']=='RC1']

In [7]:
if __name__=='__main__':
    dd.columns

In [8]:
rules = {'origin': "#driver# and #codriver# #ffo# #entrant#.",
         'ffo':['for the', 'from the', 'of the'],
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [9]:
dd.columns=[c.replace('.','') for c in dd.columns]
dd['report'] = dd.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1)


In [10]:
classes = pd.api.types.CategoricalDtype(categories = ['RC1', 'RGT', 'RC2', 'RC3', 'RC4','NRC1', 'NRC2','NRC3','NRC4'], 
                                            ordered = True)

In [11]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)
    
SUMMARY ='''# Summary

* [Entry List](entry_list.md)
'''

dd['name'] = dd['name'].astype(classes)

grouped = dd.groupby('name')

GROUP=''
for name, group in grouped:
    if len(group):
        GROUP = '## Group {}:\n'.format(name)
        #print('Group {}:'.format(name))
        entries=[]
        for i,r in group.iterrows():
            #print(r['report'])
            entries.append('{}'.format(r['report']))
        GROUP = '{}\n\n- {}'.format(GROUP,'\n- '.join(entries))
        
        fn_g = 'entries_{}.md'.format(name)
        with open('{}/{}'.format(basepath,fn_g), 'w') as out_file:
            out_file.write(GROUP)
        
        SUMMARY = '{s}\n  - [{g}]({f})'.format(s=SUMMARY,g=name, f=fn_g)

with open('{}/SUMMARY.md'.format(basepath), 'w') as out_file:
    out_file.write(SUMMARY)

In [12]:
print(SUMMARY)

# Summary

* [Entry List](entry_list.md)

  - [RC1](entries_RC1.md)
  - [RC2](entries_RC2.md)
  - [RC3](entries_RC3.md)


In [13]:
!ls report/

README.md            SS22_report.md       Section 2_report.md
SS10_report.md       SS23_report.md       Section 3_report.md
SS11_report.md       SS24_report.md       Section 4_report.md
SS12_report.md       SS2_report.md        Section 5_report.md
SS13_report.md       SS3_report.md        Section 6_report.md
SS14_report.md       SS4_report.md        Section 7_report.md
SS15_report.md       SS5_report.md        Section 8_report.md
SS16_report.md       SS6_report.md        Section 9_report.md
SS17_report.md       SS7_report.md        docs
SS18_report.md       SS8_report.md        entries_RC1.md
SS19_report.md       SS9_report.md        entries_RC2.md
SS1_report.md        SUMMARY.md           entries_RC3.md
SS20_report.md       Section 10_report.md images
SS21_report.md       Section 1_report.md  maps
